In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)

## 數據測試區

In [ ]:
df = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1_20200924 — копия.xlsx', None)
df.keys()

In [ ]:
df_sales = df['出貨明細']
df_budget = df['2020預算']

In [ ]:
df_budget.columns = ['Group', '類型', '負責業務', '預交年份', '預交月份', '本國幣別NTD']

In [ ]:
months = df_budget['預交月份'].unique().tolist()
num_month = []
for i in range(1, 13):
    num_month.append(i)
mon_dict = dict(zip(months, num_month))
mon_dict
    

In [ ]:
def transer(data):
    return mon_dict[data]

In [ ]:
df_budget['月份'] = df_budget['預交月份'].apply(transer)

In [ ]:
df_budget = df_budget.drop('預交月份', axis=1)

In [ ]:
df_budget['本國幣別NTD'] = pd.to_numeric(df_budget['本國幣別NTD'], errors='coerce')

In [ ]:
df_budget

In [ ]:
df_sales.insert(4, '類型', value='實績')

In [ ]:
df_sales_grouped = df_sales.groupby(['Group', '類型', '負責業務', '預交年份', '預交月份'])[['本國幣別NTD']].sum().reset_index()

In [ ]:
df_sales_grouped['月份'] = df_sales_grouped['預交月份'].apply(lambda x: mon_dict[x])

In [ ]:
df_sales_grouped = df_sales_grouped.drop('預交月份', axis=1)

In [ ]:
df_combined = pd.concat([df_sales_grouped, df_budget], axis=0)

In [ ]:
df_combined

In [ ]:
df_combined[df_combined['預交年份'] < 2021].groupby(['Group', '類型', '預交年份', '月份'])[['本國幣別NTD']].sum().unstack(['月份'])

## Amber 10/26要處理的數據

In [ ]:
df = pd.read_excel(r'C:\Users\kaihsu\Desktop\Robin的毛利變化 amber 20201023.xlsx', None)
# df.columns = df.columns.str.strip()
# df = df.drop(df.columns[-3:], axis=1)

In [ ]:
df = df['工作表5 (3)']

In [ ]:
column_names = df.iloc[0].tolist()

In [ ]:
df.columns = column_names

In [ ]:
df.drop(0, inplace=True)

In [ ]:
df.drop(df.index[-12:], inplace=True)

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
keep_columns = ['SupplierPN', 'Mike對應PN', 'Shipment', '出貨明細價格', 'RobinQuotation\n1022', '銀漲前成本', '銀漲後成本']
df_first = df[keep_columns]

In [ ]:
df_first['總毛利'] = df_first.apply(lambda x: (x['出貨明細價格'] - x['銀漲後成本']) * x['Shipment'] if x['出貨明細價格'] != 0 else np.nan, axis=1)
df_first['毛利率'] = df_first.apply(lambda x: (x['出貨明細價格'] - x['銀漲後成本']) / x['出貨明細價格'] if x['出貨明細價格'] != 0 else np.nan, axis=1)
df_first['Robin價格毛利率'] = df_first.apply(lambda x: (x['RobinQuotation\n1022'] - x['銀漲後成本']) / x['RobinQuotation\n1022'] if x['RobinQuotation\n1022'] !=0 else np.nan, axis=1) 
df_first['Robin總毛利'] = df_first.apply(lambda x: (x['RobinQuotation\n1022'] - x['銀漲後成本']) * x['Shipment'] if x['RobinQuotation\n1022'] != 0 else np.nan, axis=1)

In [ ]:
df_first.columns

In [ ]:
rename_columns = ['Laird PN', 'INPAQ PN', '出貨量', '出貨明細價格', 'Robin 1023單價', '銀價前成本', '銀價後成本', '總毛利', '毛利率', 'Robin毛利率', 'Robin價格總毛利']
df_first.columns = rename_columns


In [ ]:
df_first.columns

In [ ]:
final_columns = ['Laird PN', 'INPAQ PN', '出貨量', '出貨明細價格', '總毛利', '毛利率',
       'Robin 1023單價', 'Robin價格總毛利', 'Robin毛利率', '銀價前成本', '銀價後成本']
df_first = df_first.reindex(columns=final_columns)

In [ ]:
df_first.to_excel(r'C:\Users\kaihsu\Desktop\Robin毛利變化.xlsx', index=False)

In [ ]:
import os
os.listdir()

In [ ]:
df = pd.read_excel('Weekly report_v0.1.xlsx', None)
df = df['出貨明細']

In [ ]:
df.insert(10, '預交數字月份', value=df['預交日期'].dt.month)

In [ ]:
test = df.groupby(['Group','預交年份', '預交數字月份'])[['本國幣別NTD']].sum().unstack('預交數字月份').reset_index()

In [ ]:
def options(*args):
    length = len(args)
    if length == 0:
        return test
    else:
        return test[(test['Group'] == args[0]) & (test['預交年份'] == args[1])]
    

In [ ]:
options('TELTONIKA', 2020)

In [ ]:
def multi_arg(**kwargs):
    print(kwargs)
    for i in kwargs.keys():
        print(kwargs[i])

In [ ]:
multi_arg(client='Bourns', year=2020)

In [ ]:
df.isna().sum()

In [ ]:
result = df[(df['BG'] == 'RF') & (df['負責業務'] == '許凱智') & (df['預交日期'].between('2020-01-01', '2020-12-31')) & (df['狀態'] != '合約')][['Group', '預交日期','本國幣別NTD', '預交月份']]

In [ ]:
result['本國幣別NTD'].sum()

In [ ]:
result = result.reset_index(drop=True)

In [ ]:
result['本國幣別NTD'] = result['本國幣別NTD'].map(lambda x: round(x,0))

In [ ]:
dict = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8,'September':9,'October':10,'November':11,'December':12}

In [ ]:
result['預交月份'] = result['預交月份'].apply(lambda x: dict[x] if x in dict.keys() else x == 'NA')

In [ ]:
result

In [ ]:
result.groupby(['Group','預交日期'])[['本國幣別NTD']].sum()

In [ ]:
result.pivot_table(index='Group', columns='預交月份', values='本國幣別NTD').sort_values(by='預交月份', axis=1)

In [ ]:
import dataframe_image as dfi
dfi.export(result, 'result.png')

## 2021 Budget test

* 如果後續要groupby，需要groupby的欄位不可以為空值

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_final.xlsx')


In [ ]:
df2 = df[df['Unit'] == 'KPCS']
df2['Qty.'] = df2['Qty.'].apply(lambda x: x*1000)
# df2
# df3 = df[df['Unit'] == 'PCS']

In [ ]:
df3 = df[df['Unit'] == 'PCS']

In [ ]:
df2['Unit'] = df2['Unit'].str.replace('KPCS', 'PCS')
# df2

In [ ]:
df_final = pd.concat([df2, df3], axis=0)

In [ ]:
# df_final = df[df_final['Year'] == 2021].reset_index(drop=True)
df_final['Application'] = df_final['Application'].fillna('Unknown')
df_final['End customer'] = df_final['End customer'].fillna('Unknown')
df_final

In [ ]:
test = df_final.groupby(['Distributor', 'End customer', 'Product category', 'INPAQ PN', 'Application', 'Unit Price', 'Currency', 'Month'])[['Qty.']].sum().reset_index()

In [ ]:
# 重新改變columns的排列
test = test.reindex(columns=['Distributor', 'End customer', 'Product category', 'INPAQ PN', 'Application', 'Qty.', 'Unit Price', 'Currency', 'Month', 'Total NTD'])
# test.rename(columns={'Unit Price': 'Unit Price_USD/RMB'})

In [ ]:
test = pd.pivot_table(test, values='Qty.', index=['Distributor', 'End customer', 'Product category', 'INPAQ PN', 'Application', 'Unit Price', 'Currency'], columns='Month').reset_index()

In [ ]:
test

In [ ]:
test.to_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_by_Susan_format.xlsx', sheet_name='Forecast_yearly', index=False)

## 整理藝感的年度預估


In [ ]:
df = pd.read_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_藝感_response.xlsx', None)

In [ ]:
df = df['2021 Forecast']

In [ ]:
df = df.drop([0,33])

In [ ]:
df = df.drop('Customer PN', axis=1)


In [ ]:
df = df.set_index('INPAQ PN')


In [ ]:
df = df.stack()


In [ ]:
df = df.reset_index()

In [ ]:
df.to_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021預算藝感整理過.xlsx')

## 整理信昌預算


In [ ]:
df = pd.read_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_PDC_response.xlsx', None)

In [ ]:
df = df['2020出貨量']

In [ ]:
df = df.drop([0,125])

In [ ]:
df = df.drop(['Unnamed: 0', 'Unnamed: 3', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], axis=1)

In [ ]:
columns=['INPAQ PN', 'Customer PN', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
df.columns = columns

In [ ]:
df = df.set_index(['INPAQ PN', 'Customer PN'])

In [ ]:
df = df.stack()

In [ ]:
df = df.reset_index()

In [ ]:
columns = ['INPAQ PN', 'Customer PN', 'Month', 'Qty.']
df.columns = columns
df

In [ ]:
df = df.reindex(columns=df.columns.tolist() + new_columns)

In [ ]:
re_columns = ['End customer', 'Distributor', 'Year', 'Month', 'Product category', 'Sub-category', 'Customer PN', 'INPAQ PN', 'Application', 'Qty.', 'Unit', 'Unit Price', 'Currency', 'Rate', 'NTD', 'Total NTD']
df = df.reindex(columns=re_columns)


In [ ]:
df['End customer'] = df['End customer'].fillna('N/A')
df['Distributor'] = df['Distributor'].fillna('信昌')
df['Year'] = df['Year'].fillna(2021)
df['Application'] = df['Application'].fillna('N/A')
df['Unit'] = df['Unit'].fillna('PCS')


In [ ]:
df['Year'] = df['Year'].astype('int')
df.to_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_PDC_清洗完.xlsx', index=False)